In [1]:
import sklearn.datasets
import sklearn.utils
iris = sklearn.datasets.load_iris()
X_all, y_all = sklearn.utils.shuffle(iris.data, iris.target, random_state=42)
holdout_size = 30
X_train, y_train = X_all[holdout_size:], y_all[holdout_size:]
X_test, y_test = X_all[:holdout_size], y_all[:holdout_size]
print('expected  {}'.format(y_test))

expected  [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


In [2]:
from lale.lib.sklearn import LogisticRegression as LR
from lale.helpers import print_yaml
print_yaml('hyperparameter defaults', LR.hyperparam_defaults())

hyperparameter defaults:
  C: 1.0
  class_weight: null
  dual: false
  fit_intercept: true
  intercept_scaling: 1.0
  max_iter: 100
  multi_class: auto
  n_jobs: null
  penalty: l2
  random_state: null
  solver: liblinear
  tol: 0.0001
  verbose: 0
  warm_start: false


In [3]:
print_yaml('schema of solver hyperparameter', LR.hyperparam_schema('solver'))

schema of solver hyperparameter:
  default: liblinear
  description: Algorithm for optimization problem.
  enum:
  - newton-cg
  - lbfgs
  - liblinear
  - sag
  - saga


In [4]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
trainable_lr = LR(LR.solver.lbfgs, LR.multi_class.multinomial, random_state=42)
trained_lr = trainable_lr.fit(X_train, y_train)
print('predicted {}'.format(trained_lr.predict(X_test)))

predicted [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


In [5]:
from lale.helpers import assert_raises_validation_error
with assert_raises_validation_error():
    LR(C=0.0)

error:
  message: 0.0 is less than or equal to the minimum of 0.0
  schema:
    default: 1.0
    description: Inverse regularization strength. Smaller values specify stronger
      regularization.
    distribution: loguniform
    exclusiveMinimum: true
    maximumForOptimizer: 32768
    minimum: 0.0
    minimumForOptimizer: 0.03125
    type: number


In [6]:
with assert_raises_validation_error():
    LR(LR.solver.sag, LR.penalty.l1)

error:
  message: '{''solver'': ''sag'', ''penalty'': ''l1'', ''dual'': False, ''C'': 1.0,
    ''tol'': 0.0001, ''fit_intercept'': True, ''intercept_scaling'': 1.0, ''class_weight'':
    None, ''random_state'': None, ''max_iter'': 100, ''multi_class'': ''auto'', ''verbose'':
    0, ''warm_start'': False, ''n_jobs'': None} is not valid under any of the given
    schemas'
  schema:
    anyOf:
    - properties:
        solver:
          not:
            enum:
            - newton-cg
            - sag
            - lbfgs
      type: object
    - properties:
        penalty:
          enum:
          - l2
      type: object
    description: The newton-cg, sag, and lbfgs solvers support only l2 penalties.


In [7]:
with assert_raises_validation_error():
    LR(LR.penalty.l2, LR.solver.sag, dual=True)

error:
  message: '{''penalty'': ''l2'', ''solver'': ''sag'', ''dual'': True, ''C'': 1.0,
    ''tol'': 0.0001, ''fit_intercept'': True, ''intercept_scaling'': 1.0, ''class_weight'':
    None, ''random_state'': None, ''max_iter'': 100, ''multi_class'': ''auto'', ''verbose'':
    0, ''warm_start'': False, ''n_jobs'': None} is not valid under any of the given
    schemas'
  schema:
    anyOf:
    - properties:
        dual:
          enum:
          - false
      type: object
    - properties:
        penalty:
          enum:
          - l2
        solver:
          enum:
          - liblinear
      type: object
    description: The dual formulation is only implemented for l2 penalty with the
      liblinear solver.


In [8]:
with assert_raises_validation_error():
    dummy = trainable_lr.fit(y_train, X_train)

AssertionError: 

In [9]:
from lale.helpers import assert_warns
with assert_warns(DeprecationWarning):
    trainable_lr.predict(X_test)

{error: 'The `predict` method is deprecated on a trainable operator, because the learned
    coefficients could be accidentally overwritten by retraining. Call `predict` on
    the trained operator returned by `fit` instead.'}
